In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mlub2021-session10/y_train.p
/kaggle/input/mlub2021-session10/X_test.p
/kaggle/input/mlub2021-session10/X_unlabeled.p
/kaggle/input/mlub2021-session10/X_train.p


In [2]:
# others imports
import pickle
import imageio
from skimage import transform,io
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import BatchNormalization
from keras.layers import Dropout, Flatten, Conv2D, MaxPool2D, Dense, Conv2DTranspose, UpSampling2D

In [3]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [4]:
X = pickle.load(open("../input/mlub2021-session10/X_train.p", "rb"))
Y = pickle.load(open("../input/mlub2021-session10/y_train.p", "rb"))
X_unlabeled = pickle.load(open("../input/mlub2021-session10/X_unlabeled.p", "rb"))
X_test = pickle.load(open("../input/mlub2021-session10/X_test.p", "rb"))

In [5]:
X_train, X_val, y_train, y_val= train_test_split(X, Y, test_size=0.2, random_state=66)

In [6]:
X_train.shape

(4000, 32, 32, 3)

In [7]:
X_train = X_train.astype('float32') / 255.
X_val = X_val.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.
X_unlabeled = X_unlabeled.astype('float32') / 255.

In [8]:
# define encoder 
# reused network from task Learning to count
encoder = keras.models.Sequential()
encoder.add(Conv2D(filters=32, kernel_size=5, padding="same", activation="relu", input_shape=[32, 32, 3]))
encoder.add(MaxPool2D(pool_size=2))

encoder.add(Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"))
encoder.add(MaxPool2D(pool_size=2))

encoder.add(Conv2D(filters=128, kernel_size=3, padding="same", activation="relu"))
encoder.add(MaxPool2D(pool_size=2))

2022-01-12 11:39:05.323260: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-12 11:39:05.477477: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-12 11:39:05.478618: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-12 11:39:05.480480: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [9]:
encoder.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        2432      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 128)         73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 128)         0         
Total params: 94,784
Trainable params: 94,784
Non-trainable params: 0
____________________________________________________

In [10]:
# define decoder
decoder = keras.models.Sequential()
decoder.add(Conv2DTranspose(64, kernel_size=5, padding="same", activation="relu", input_shape=[4, 4, 128]))
decoder.add(UpSampling2D((2,2)))
decoder.add(Conv2DTranspose(32, kernel_size=3, padding="same", activation="relu"))
decoder.add(UpSampling2D((4,4)))
decoder.add(Conv2DTranspose(3, kernel_size=3, padding="same", activation="sigmoid"))

In [11]:
decoder.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_transpose (Conv2DTran (None, 4, 4, 64)          204864    
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 8, 8, 32)          18464     
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 32, 32, 3)         867       
Total params: 224,195
Trainable params: 224,195
Non-trainable params: 0
_________________________________________________________________


In [12]:
# define autoencoder
autoencoder = keras.models.Sequential([encoder, decoder])
autoencoder.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 4, 4, 128)         94784     
_________________________________________________________________
sequential_1 (Sequential)    (None, 32, 32, 3)         224195    
Total params: 318,979
Trainable params: 318,979
Non-trainable params: 0
_________________________________________________________________


In [13]:
adam = tf.keras.optimizers.Adam(learning_rate = 0.0001, beta_1 = 0.9, beta_2 = 0.999, amsgrad = True)
autoencoder.compile(loss="mean_squared_error", optimizer="Adam", metrics=['accuracy'])

In [14]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)
check_pointer = ModelCheckpoint(filepath = "my_keras_model.h5", save_best_only=True)

In [15]:
auto = autoencoder.fit(X_unlabeled, X_unlabeled, epochs=100,
          validation_data=(X_val, X_val), callbacks=[early_stopping_cb, check_pointer])

2022-01-12 11:39:10.681015: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100


2022-01-12 11:39:12.546692: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


1407/1407 [==============================] - 17s 7ms/step - loss: 0.0140 - accuracy: 0.7096 - val_loss: 0.0104 - val_accuracy: 0.7319
Epoch 2/100
1407/1407 [==============================] - 9s 6ms/step - loss: 0.0090 - accuracy: 0.7624 - val_loss: 0.0080 - val_accuracy: 0.7724
Epoch 3/100
1407/1407 [==============================] - 10s 7ms/step - loss: 0.0078 - accuracy: 0.7670 - val_loss: 0.0081 - val_accuracy: 0.7760
Epoch 4/100
1407/1407 [==============================] - 9s 6ms/step - loss: 0.0071 - accuracy: 0.7716 - val_loss: 0.0067 - val_accuracy: 0.7864
Epoch 5/100
1407/1407 [==============================] - 10s 7ms/step - loss: 0.0066 - accuracy: 0.7741 - val_loss: 0.0065 - val_accuracy: 0.7999
Epoch 6/100
1407/1407 [==============================] - 9s 7ms/step - loss: 0.0064 - accuracy: 0.7771 - val_loss: 0.0061 - val_accuracy: 0.8038
Epoch 7/100
1407/1407 [==============================] - 10s 7ms/step - loss: 0.0061 - accuracy: 0.7796 - val_loss: 0.0060 - val_accuracy: 

In [16]:
num_layers = encoder.layers

In [17]:
# use encoder as network model
encoder.add(Flatten())
encoder.add(Dense(512, activation="relu"))
encoder.add(Dense(256, activation="relu"))
encoder.add(Dense(128, activation="relu"))
encoder.add(Dense(100, activation='softmax'))

In [18]:
final_model = encoder

In [19]:
final_model.compile(loss="sparse_categorical_crossentropy", optimizer="Adam", metrics=['accuracy'])

In [20]:
res = final_model.fit(X_train, y_train, epochs=200, validation_data=(X_val, y_val), callbacks=[early_stopping_cb, check_pointer])

Epoch 1/200
125/125 [==============================] - 1s 6ms/step - loss: 4.5787 - accuracy: 0.0250 - val_loss: 4.4884 - val_accuracy: 0.0370
Epoch 2/200
125/125 [==============================] - 1s 5ms/step - loss: 4.2814 - accuracy: 0.0430 - val_loss: 4.2016 - val_accuracy: 0.0640
Epoch 3/200
125/125 [==============================] - 1s 5ms/step - loss: 3.9976 - accuracy: 0.0760 - val_loss: 4.0108 - val_accuracy: 0.0880
Epoch 4/200
125/125 [==============================] - 1s 6ms/step - loss: 3.7375 - accuracy: 0.1042 - val_loss: 3.9405 - val_accuracy: 0.1030
Epoch 5/200
125/125 [==============================] - 1s 8ms/step - loss: 3.5023 - accuracy: 0.1412 - val_loss: 3.8417 - val_accuracy: 0.1110
Epoch 6/200
125/125 [==============================] - 1s 7ms/step - loss: 3.2255 - accuracy: 0.1842 - val_loss: 3.8289 - val_accuracy: 0.1230
Epoch 7/200
125/125 [==============================] - 1s 6ms/step - loss: 2.9391 - accuracy: 0.2442 - val_loss: 3.8737 - val_accuracy: 0.1520

In [21]:
y_test = final_model.predict(X_test)
y_test = [np.argmax(x) for x in y_test]

df_output = pd.DataFrame(y_test)
df_output.index.name = 'index'
df_output.columns = ['label']
df_output.to_csv('output.csv')